# Classification with PyTorch and Petastorm

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from pyspark.sql import DataFrame
from plantclef.utils import get_spark
from pyspark.sql import functions as F


spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 01:22:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/21 01:22:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# paths
gcs_path = "gs://dsgt-clef-plantclef-2024/data/process"
embedding_path = "training_cropped_resized_v2/dino_dct/data"
# data module parameters
input_path = f"{gcs_path}/{embedding_path}"
feature_col = "dct_embedding"
limit_species = 5
species_image_count = 100
batch_size = 32
num_partitions = 32
workers_count = 16

In [4]:
from plantclef.baseline.data import PetastormDataModule

# test data module
data_module = PetastormDataModule(
    spark,
    input_path,
    feature_col,
    limit_species,
    species_image_count,
    batch_size,
    num_partitions,
)
data_module.setup()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
Converting floating-point columns to float32
The median size 8434 B (< 50 MB) of the parquet files is too small. Total size: 272576 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///mnt/data/

In [5]:
# model parameters
num_features = int(
    len(data_module.train_data.select("dct_embedding").first()["dct_embedding"])
)
num_classes = int(data_module.train_data.select("species_id").distinct().count())
print(f"num_features: {num_features}  num_classes: {num_classes}")

num_features: 64  num_classes: 5


In [6]:
from plantclef.baseline.model import LinearClassifier

# test classifier
model = LinearClassifier(
    num_features,
    num_classes,
)

In [7]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

# train model
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    reload_dataloaders_every_n_epochs=1,
    default_root_dir="gs://dsgt-clef-plantclef-2024/models/torch-petastorm-v1",
    callbacks=[
        EarlyStopping(monitor="val_loss", mode="min"),
        ModelCheckpoint(monitor="val_loss", save_last=True),
    ],
)
# fit model
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mgustine/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: gs://dsgt-clef-plantclef-2024/models/torch-petastorm-v1/lightning_logs
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
The m

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mgustine/.local/lib/python3.10/site-packages/petastorm/etl/dataset_metadata.py:402: FutureWarning: Specifying the 'metadata_nthreads' argument is deprecated as of pyarrow 8.0.0, and the argument will be removed in a future version
  dataset = pq.ParquetDataset(path_or_paths, filesystem=fs, validate_schema=False, metadata_nthreads=10)
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/etl/dataset_metadata.py:362: FutureWarning: 'ParquetDataset.common_metadata' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version.
  if not dataset.common_metadata:
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/reader.py:420: FutureWarning: Specifying the 'metadata_nthreads' argument is deprecated as of pyarrow 8.0.0, and the argument will be removed in a future version
  self.dataset = pq.ParquetDataset(dataset_path, filesystem=pyarrow_filesystem,
/home/mgustine/.local/lib/python3.10/site-packages/petastorm/unischema.py:317: FutureWarning: 'Par

Epoch 9: |          | 29/? [00:00<00:00, 35.55it/s, v_num=0, train_loss=112.0, val_loss=160.0]     

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: |          | 29/? [00:01<00:00, 18.81it/s, v_num=0, train_loss=112.0, val_loss=160.0]
